# Stream Ciphers

treats the plaintext as a stream and encrypts the bits individually

smaller and faster 

assumed to be more efficient

encryption and decryption are the same procedure -XOR

they can reach very high level of secrecy

One time pod: key as long as the plaintext, uniformly distributed in the key space, key must be used only once -> information theory-wise secure

Security in stream ciphers relies on key stream

OTP is unpractical because stream ciphers rely on random number generators

-maybe talk a bit about RNGs- 

RNG must be reproducible and unpredictable



In [1]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

In [2]:
random = [1, 0, 1, 0, 1, 0, 1, 0]
bits = Bits(random)
bits

Bits([True, False, True, False, True, False, True, False])

## LFSR

one of the main building blocks of PRNGs

In [3]:
lfsr = LFSR({1, 2, 5}, 0b000)
bits = lfsr.cycle()
print(bits)

0


In [4]:
lfsr = LFSR({1, 2, 5}, 0b00101)
output = lfsr.run_steps(10)

In [5]:
lfsr.__next__()

False

In [6]:
output

Bits([False, True, False, False, True, True, True, False, True, False])

In [7]:
bits = lfsr.cycle(state=0b00101)
bits

Bits([False, True, False, False, True, True, True])

## Berlekamp-Massey Algorithm

binary output sequence -> shortest lfsr able to produce the sequence

exploits the ptoperty that x p_i+b[t-i] must be zero

makes the system prone to KPA attck. If eve knows enough x_i and y_i's she can compute b_i's and apply B-M algorithm to infer P(X)

In [8]:
test_bits = Bits([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1])
print(berlekamp_massey(test_bits))


{0, 3, 5}


In [9]:
with open('binary_sequence.bin', 'rb') as f:
        binary_sequence = f.read()

In [10]:
binary_sequence[:50]

b'\xbb`\xef\x067\xae\xd0K"Vd]#Q\xeb\x02~<\xe6C\xbe\xed5\xd0\xec\xada\xe8\x89h\xf3\xbdFc\x96\xb5\x8e\xb0\x03\xabVFY#\xd1\xeb">\xb5\xe4'

In [11]:
bits = Bits(binary_sequence)

In [12]:
poly = berlekamp_massey(bits)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 18, 7}
Linear complexity: 18


## Alternating Step Generator

In [13]:
alt_step = AlternatingStep()

In [14]:
bits = alt_step.run(25)

t=0   LFSRC=11111 bc=True   LFSR0=111 b0=True   LFSR1=1111 b1=True   output=None
t=1   LFSRC=01111 bc=True   LFSR0=111 b0=True   LFSR1=0111 b1=True   output=False
t=2   LFSRC=00111 bc=True   LFSR0=111 b0=True   LFSR1=1011 b1=True   output=False
t=3   LFSRC=10011 bc=True   LFSR0=111 b0=True   LFSR1=0101 b1=True   output=False
t=4   LFSRC=11001 bc=True   LFSR0=111 b0=True   LFSR1=1010 b1=False   output=True
t=5   LFSRC=01100 bc=False   LFSR0=011 b0=True   LFSR1=1010 b1=False   output=True
t=6   LFSRC=10110 bc=False   LFSR0=101 b0=True   LFSR1=1010 b1=False   output=True
t=7   LFSRC=01011 bc=True   LFSR0=101 b0=True   LFSR1=1101 b1=True   output=False
t=8   LFSRC=00101 bc=True   LFSR0=101 b0=True   LFSR1=0110 b1=False   output=True
t=9   LFSRC=10010 bc=False   LFSR0=010 b0=False   LFSR1=0110 b1=False   output=False
t=10   LFSRC=01001 bc=True   LFSR0=010 b0=False   LFSR1=0011 b1=True   output=True
t=11   LFSRC=00100 bc=False   LFSR0=001 b0=True   LFSR1=0011 b1=True   output=False
t=12   LF